In [17]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import random
import time

In [67]:
# simplified form from Appendix A of 2410.21260

def evolve_Nc(delta, eta_max, ICs = "ones"):
    nsteps = round(eta_max/delta)
    d2 = delta**2
    
    G = np.ones((nsteps + 1, nsteps + 1))
    G2 = np.ones((nsteps + 1, nsteps + 1))
    Gm = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

    I3 = np.ones((nsteps + 1, nsteps + 1))
    I4 = np.ones((nsteps + 1, nsteps + 1))
    I5 = np.ones((nsteps + 1, nsteps + 1))
    It = np.ones((nsteps + 1, nsteps + 1))
    Gm3 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    
    if type(ICs) != str:
        for amp, ne_amp, ic in zip([G, G2, I3, I4, I5], [Gm, Gm2, Gm3, Gm4, Gm5], ICs):
            amp *= ic
            ne_amp *= ic
        It *= ICs[5]
        
    print("amplitudes initialized to", ICs)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            G[i, j] = G[i, j-1]
            G[i, j] += d2*np.sum(Gm[i, i:j, j-1] + 3*G[i:j, j-1] + 2*G2[i:j, j-1] + 2*Gm2[i, i:j, j-1])
            
            Gm[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                Gm[i, k, j] = Gm[i, k-1, j-1]
                Gm[i, k, j] += d2*np.sum(Gm[i, k-1:j, j-1] + 3*G[k-1:j, j-1] + 2*G2[k-1:j, j-1] + 2*Gm2[i, k-1:j, j-1])

            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(G[:i, j-i:j] + 2*G2[:i, j-i:j])

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(2*Gm3[i, i:j, j-1] - 4*Gm4[i, i:j, j-1] + 2*Gm5[i, i:j, j-1] - 2*Gm2[i, i:j, j-1])
            I3[i, j] += 0.5*d2*np.trace(4*I3[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j])

            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1] 
                Gm3[i, k, j] += 0.5*d2*np.sum(2*Gm3[i, k-1:j, j-1] - 4*Gm4[i, k-1:j, j-1] + 2*Gm5[i, k-1:j, j-1] - 2*Gm2[i, k-1:j, j-1])
            
            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 2*G[:i, j-i:j] - 3*G2[:i, j-i:j])
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(-2*I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 4*G[:i, j-i:j] + 7*G2[:i, j-i:j])
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(G[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j])
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [G, G2, I3, I4, I5, It]
    

In [75]:
# large Nc 
eta_maxs = {0.0125:10, 0.016:10, 0.025:20, 0.032:20, 0.0375:30, 0.05:40, 0.0625:50, 0.075:60, 0.08:60, 0.1:70}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/"

for delta, eta_max in eta_maxs.items():
    # if delta != 0.05: continue 
    # eta_max = 4 # testing
    eta_max = 300*delta # limit to 300 steps for fast data

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer") 
        break
        
    print("nsteps:", round(eta_max/delta)) 
    delta_str = str(delta)[2:]
    amp_strs = {0:"G", 1:"G2", 2:"I3", 3:"I4", 4:"I5", 5:"It"}
    print("Staring (d, eta_max)=", delta, eta_max)
    
    output = evolve_Nc(delta, eta_max, ICs = ICs)

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+"d"+delta_str+"_"+ICs_tag)
    for iamp in range(6):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_Nc_"+ICs_tag+"_"+amp_strs[iamp]+".dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0
    

nsteps: 300
Staring (d, eta_max)= 0.0125 3.75
amplitudes initialized to ones
10.0% done (17:23.02)
20.0% done (17:23.02)
30.0% done (17:23.03)
40.0% done (17:23.06)
50.0% done (17:23.10)
60.0% done (17:23.17)
70.0% done (17:23.26)
80.0% done (17:23.39)
90.0% done (17:23.55)
100.0% done (17:24.16)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/
nsteps: 300
Staring (d, eta_max)= 0.016 4.8
amplitudes initialized to ones
10.0% done (17:24.17)
20.0% done (17:24.17)
30.0% done (17:24.19)
40.0% done (17:24.21)
50.0% done (17:24.25)
60.0% done (17:24.32)
70.0% done (17:24.41)
80.0% done (17:24.54)
90.0% done (17:25.10)
100.0% done (17:25.31)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc/
nsteps: 300
Staring (d, eta_max)= 0.025 7.5
amplitudes initialized to ones
10.0% done (17:25.31)
20.0% done (17:25.32)
30.0% done (17:25.33)
40.0% done (17:25.36)
50.0% done (17:25.40)
60.0% done (17:25.47)
7

In [83]:
# here lambda is a cutoff, also includes Qt corrections from 2406.11647 and It amplitude from 2410.21260

def evolve_NcNf(delta, eta_max, Nf=3.0, ICs="ones"):
    Nr = 0.25*(Nf/3.0)
    nsteps = round(eta_max/delta)
    d2 = delta**2
    
    G = np.ones((nsteps + 1, nsteps + 1))
    Q = np.ones((nsteps + 1, nsteps + 1))
    G2 = np.ones((nsteps + 1, nsteps + 1))
    Qt = np.ones((nsteps + 1, nsteps + 1))
    GmT = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    GmB = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm2 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

    I3t = np.ones((nsteps + 1, nsteps + 1))
    I3 = np.ones((nsteps + 1, nsteps + 1))
    I4 = np.ones((nsteps + 1, nsteps + 1))
    I5 = np.ones((nsteps + 1, nsteps + 1))
    It = np.ones((nsteps + 1, nsteps + 1))
    Gm3t = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm3 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm4 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))
    Gm5 = np.ones((nsteps + 1, nsteps + 1, nsteps + 1))

            
    if type(ICs) != str:  
        for amp, ne_amp, ic in zip([G, Q, G2, I3t, I3, I4, I5], [GmT, GmB, Gm2, Gm3t, Gm3, Gm4, Gm5], ICs):
            amp *= ic
            ne_amp *= ic
            
    elif "basis" in ICs:
        pass
        # FIXME: add support for basis functions
    
    # basis_func = (a - eta), (b - s_10), (c - const)
    # basis_amp = 0 - 7 (G, Q, G2, I3t, I3, I4, I5, I6)
    
    # amps = [G, Q, G2, I3t, I3, I4, I5, I6]
    # neighs = [GmT, GmB, Gm2, Gm3t, Gm3, Gm4, Gm5, Gm6]
    
    # if basis_func == "c":
    #     amps[basis_amp] += 1
    #     neighs[basis_amp] += 1
        
    # elif basis_func == "b":
    #     amps[basis_amp] +=
    #         if ia != basis_amp: continue
    #         amp += 1
            
            
    #     amp = np.array([[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
    #     Gm = np.array([[[coef * ((2 * (i - n_steps+j)) + ((CF - 2) * j)) for j in range(n_steps + 1)] for k in range(2*n_steps + 1)] for i in range(n_steps + 1)])

     
    print("amplitudes initialized to", ICs)
    print("Nf=", Nf)
        
    for j in range(1, nsteps + 1):
        for i in range(j):
            
            Q[i, j] = Q[i, j-1]
            Q[i, j] += 0.5*d2*np.sum(Q[i:j-1, j-1] + 2*G2[i:j-1, j-1]) 
            Q[i, j] += 0.5*d2*np.trace(Q[:i, j-i:j] + 2*G2[:i, j-i:j]) 
            Q[i, j] += d2*np.sum(2*G[i:j-1,j-1] + Q[i:j-1,j-1] + 2*G2[i:j-1,j-1] + 2*GmT[i, i:j-1, j-1] - GmB[i, i:j-1, j-1] + 2*Gm2[i, i:j-1, j-1])
            
            GmB[i, i, j] = Q[i, j]
            for k in range(i+1, j+1):
                GmB[i, k, j] = GmB[i, k-1, j-1]
                GmB[i, k, j] += 0.5*d2*np.sum(Q[k-1:j-1, j-1] + 2*G2[k-1:j-1, j-1])
                ii = max(i, k-1)
                GmB[i, k, j] += d2*np.sum(2*GmT[i, ii:j-1, j-1] - GmB[i, ii:j-1, j-1] + 2*Gm2[i, ii:j-1, j-1] + 2*G[ii:j-1, j-1] + Q[ii:j-1, j-1] + 2*G2[ii:j-1, j-1])
            
            G[i, j] = G[i, j-1]
            G[i, j] -= Nr*d2*np.trace(Q[:i, j-i:i] + 2*G2[:i, j-i:i])
            G[i, j] += d2*np.sum(3*G[i:j-1, j-1] + 2*G2[i:j-1, j-1] - 2*Nr*Qt[i:j-1, j-1])
            G[i, j] += d2*np.sum(GmT[i, i:j-1, j-1] + (2-2*Nr)*Gm2[i, i:j-1, j-1] - Nr*GmB[i, i:j-1, j-1])
            
            GmT[i, i, j] = G[i, j]
            for k in range(i+1, j+1):
                GmT[i, k, j] = GmT[i, k-1, j-1]
                ii = max(i, k-1)
                GmT[i, k, j] += d2*np.sum(GmT[i, ii:j-1, j-1] + (2-2*Nr)*Gm2[i, ii:j-1, j-1] - Nr*GmB[i, ii:j-1, j-1] + 3*G[ii:j-1, j-1] + 2*G2[ii:j-1, j-1] - 2*Nr*Qt[ii:j-1, j-1])
            
            G2[i, j] = G2[i, j-1]
            G2[i, j] += 2*d2*np.trace(G[:i, j-i:j] + 2*G2[:i, j-i:j])

            Gm2[i, i, j] = G2[i, j]
            Gm2[i, i+1:j+1, j] = Gm2[i, i:j, j-1]

            Qt[i, j] = Qt[i, j-1]
            Qt[i, j] -= d2*np.trace(Q[:i, j-i:j] + 2*G2[:i, j-i:j])
              
            I3[i, j] = I3[i, j-1]
            I3[i, j] += 0.5*d2*np.sum(4*Gm3t[i, i:j-1, j-1] - 2*Gm3[i, i:j-1, j-1] - 4*Gm4[i, i:j-1, j-1] + 2*Gm5[i, i:j-1, j-1] - 2*Gm2[i, i:j-1, j-1])
            I3[i, j] += 0.5*d2*np.trace(4*I3t[:i, j-i:j] - 4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] - 4*G[:i, j-i:j] - 6*G2[:i, j-i:j])
            
            Gm3[i, i, j] = I3[i, j]
            for k in range(i+1, j+1):
                Gm3[i, k, j] = Gm3[i, k-1, j-1]
                ii = max(i, k-1)
                Gm3[i, k, j] += d2*np.sum(2*Gm3t[i, ii:j-1, j-1] - Gm3[i, ii:j-1, j-1] - 2*Gm4[i, ii:j-1, j-1] + Gm5[i, ii:j-1, j-1] - Gm2[i, ii:j-1, j-1])
            
            I3t[i, j] = I3t[i, j-1]
            I3t[i, j] += 0.5*d2*np.sum(2*Gm3t[i, i:j-1, j-1] - 2*Nr*Gm3[i, i:j-1, j-1] + (4*Nr - 4)*Gm4[i, i:j-1, j-1] + (2 - 2*Nr)*Gm5[i, i:j-1, j-1] + (2*Nr - 2)*Gm2[i, i:j-1, j-1])
            I3t[i, j] += 0.5*d2*np.trace(-2*Nr*I3[:i, j-i:j] + 4*I3t[:i, j-i:j] + (4*Nr-4)*I4[:i, j-i:j] + (2 - 2*Nr)*I5[:i, j-i:j] - 4*G[:i, j-i:j] + (2*Nr - 6)*G2[:i, j-i:j])
            
            Gm3t[i, i, j] = I3t[i, j]
            for k in range(i+1, j+1):
                Gm3t[i, k, j] = Gm3t[i, k-1, j-1]
                Gm3t[i, k, j] += d2*np.sum(Gm3t[i, ii:j-1, j-1] - Nr*Gm3[i, ii:j-1, j-1] + (2*Nr - 2)*Gm4[i, ii:j-1, j-1] + (1 - Nr)*Gm5[i, ii:j-1, j-1] + (Nr - 1)*Gm2[i, ii:j-1, j-1])

            I4[i, j] = I4[i, j-1]
            I4[i, j] += 0.5*d2*np.trace(4*I4[:i, j-i:j] + 2*I5[:i, j-i:j] + G2[:i, j-i:j])
            
            Gm4[i, i, j] = I4[i, j]
            Gm4[i, i+1:j+1, j] = Gm4[i, i:j, j-1]
            
            I5[i, j] = I5[i, j-1]
            I5[i, j] += 0.5*d2*np.trace(-2*I3t[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j] + 2*G[:i, j-i:j] + 3*G2[:i, j-i:j])
            
            Gm5[i, i, j] = I5[i, j]
            Gm5[i, i+1:j+1, j] = Gm5[i, i:j, j-1]

            It[i, j] = It[i, j-1]
            It[i, j] -= 0.5*d2*np.trace(Q[:i, j-i:j] + 3*G2[:i, j-i:j] - I3[:i, j-i:j] + 2*I4[:i, j-i:j] - I5[:i, j-i:j])
                
        if(np.mod(j, round(nsteps/10)) == 0): 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")

    return [Q, G, G2, Qt, I3, I3t, I4, I5, It]
    

In [87]:
# large Nc&Nf (can handle 6/9 = .6 of the memory of large Nc)
eta_maxs = {0.0125:6, 0.016:6, 0.025:12, 0.032:12, 0.0375:18, 0.05:24, 0.0625:30, 0.075:36, 0.08:36, 0.1:42}

#     ICs = np.random.randint(-6,6, size=(6))
ICs = "ones"
ICs_tag = "ones"
output_dir = "/Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/"

Nf = 3

for delta, eta_max in eta_maxs.items():
    # if delta != 0.05: continue 
    # eta_max = 4 # testing
    eta_max = 300*delta # limit to 300 steps for fast data

    if round(eta_max/delta) > 800:
        print("Error: nsteps > 800 and will probably crash your computer") 
        break
        
    print("nsteps:", round(eta_max/delta)) 
    delta_str = str(delta)[2:]
    amp_strs = {0:"Q", 1:"G", 2:"G2", 3:"Qt", 4:"I3", 5:"I3t", 6:"I4", 7:"I5", 8:"It"}
    print("Staring (d, eta_max)=", delta, eta_max)
    
    output = evolve_NcNf(delta, eta_max, ICs = ICs)

    # save output
    if not os.path.exists(output_dir+"d"+delta_str+"_"+ICs_tag): os.mkdir(output_dir+"d"+delta_str+"_"+ICs_tag)
    for iamp in range(9):
        np.savetxt(output_dir+"d"+delta_str+"_"+ICs_tag+"/d"+delta_str+"_NcNf"+str(Nf)+"_"+ICs_tag+"_"+amp_strs[iamp]+".dat", output[iamp])
    print("wrote out amplitudes to", output_dir)
    output = 0
    

nsteps: 300
Staring (d, eta_max)= 0.0125 3.75
amplitudes initialized to ones
Nf= 3.0
10.0% done (19:45.29)
20.0% done (19:45.30)
30.0% done (19:45.33)
40.0% done (19:45.39)
50.0% done (19:45.48)
60.0% done (19:46.03)
70.0% done (19:46.23)
80.0% done (19:46.50)
90.0% done (19:47.26)
100.0% done (19:48.12)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 300
Staring (d, eta_max)= 0.016 4.8
amplitudes initialized to ones
Nf= 3.0
10.0% done (19:48.13)
20.0% done (19:48.14)
30.0% done (19:48.16)
40.0% done (19:48.22)
50.0% done (19:48.31)
60.0% done (19:48.45)
70.0% done (19:49.05)
80.0% done (19:49.32)
90.0% done (19:50.08)
100.0% done (19:50.54)
wrote out amplitudes to /Users/brandonmanley/Desktop/PhD/moment_evolution/evolved_dipoles/largeNc&Nf/
nsteps: 300
Staring (d, eta_max)= 0.025 7.5
amplitudes initialized to ones
Nf= 3.0
10.0% done (19:50.55)
20.0% done (19:50.56)
30.0% done (19:50.59)
40.0% done (19:51.04)
50.0% done (19:

In [ ]:
# here, lambda is a scale characterizing the target (not a cutoff)

def evolve_NcNf_nocutoff(delta, eta_max, Nf=3.0, IC = "ones", verb=True):
    nsteps = round(eta_max/delta)
    d = delta**2
    
    Nc = 3.0
    Nratio = Nf/Nc
    
    if IC == "ones":
        Q = np.ones((nsteps + 1, nsteps + 1))
        G = np.ones((nsteps + 1, nsteps + 1))
        G2 = np.ones((nsteps + 1, nsteps + 1))
        I3 = np.ones((nsteps + 1, nsteps + 1))
        I3t = np.ones((nsteps + 1, nsteps + 1))
        I4 = np.ones((nsteps + 1, nsteps + 1))
        I5 = np.ones((nsteps + 1, nsteps + 1))
        I6 = np.ones((nsteps + 1, nsteps + 1))
        
        Gb = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm2 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm3 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm3t = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm4 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm5 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
        Gm6 = np.ones((nsteps + 1, 2*nsteps + 1, nsteps + 1))
    print('Finished initialization')
    
    for j in range(1, nsteps + 1):
        for i in range(j - nsteps, j):
            
            if i < 0: continue
            
            li = i-j+nsteps
            pli = i-(j-1)+nsteps
            
            # increment Q
            Q[li, j] = Q[pli, j-1]  
            Q[li, j] += d*np.sum(2*Gm[pli, i+nsteps:j-1+nsteps, j-1] + 2*G[pli:nsteps, j-1] + Q[pli:nsteps, j-1] - Gb[pli, i+nsteps:j-1+nsteps, j-1] + 2*Gm2[pli, i+nsteps:j-1+nsteps, j-1] + 2*G2[pli:nsteps, j-1])
            Q[li, j] += (d/2)*np.sum(Q[li:nsteps, j-1] + 2*G2[li:nsteps, j-1])
            Q[li, j] += (d/2)*np.sum(Q[li, :j-1] + 2*G2[li, :j-1])

             # increment Gmb (Gamma bar)
            Gb[li, i+nsteps, j] = Q[li, j]
            for k in range(i+1, j+1):
                Gb[li, k+nsteps, j] = Gb[pli, k-1+nsteps, j-1]
                Gb[li, k+nsteps, j] += (d/2)*np.sum(Q[k-j+nsteps:nsteps, j-1] + 2*G2[k-j+nsteps:nsteps, j-1])
                lb = max(i, k-1)
                Gb[li, k+nsteps, j] += d*np.sum(2*Gm[pli, lb+nsteps:j-1+nsteps, j-1] + 2*G[lb+nsteps-j+1:nsteps, j-1] + Q[lb+nsteps-j+1:nsteps, j-1] - Gb[pli, lb+nsteps:j-1+nsteps, j-1] + 2*Gm2[pli, lb+nsteps:j-1+nsteps, j-1] + 2*G2[lb+nsteps-j+1:nsteps, j-1])
                 
            # increment G 
            G[li, j] = G[pli, j-1] 
            G[li, j] += -(d/4)*(Nratio)*np.sum(Q[li, :j] + 2*G2[li, :j])
            G[li, j] += d*np.sum(Gm[pli, i+nsteps:j-1+nsteps, j-1] + 3*G[pli:nsteps, j-1] + 2*G2[pli:nsteps, j-1] + ((4 - Nratio)/2) * Gm2[pli, i+nsteps:j-1+nsteps, j-1] - (Nratio/4)*Gb[pli, i+nsteps:j-1+nsteps, j-1])
                
            # increment Gmt (Gamma tilde) 
            Gm[li, i+nsteps, j] = G[li, j]
            for k in range(i+1, j+1):
                Gm[li, k+nsteps, j] = Gm[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm[li, k+nsteps, j] += d*np.sum(Gm[pli, lb+nsteps:j-1+nsteps, j-1] + 3*G[lb+nsteps-j+1:nsteps, j-1] + 2*G2[lb+nsteps-j+1:nsteps, j-1] + ((4 - Nratio)/2)*Gm2[pli, lb+nsteps:j-1+nsteps, j-1] - (Nratio/4)*Gb[pli, lb+nsteps:j-1+nsteps, j-1])

            # increment G2
            G2[li, j] = G2[pli, j-1]
            G2[li, j] += 2*d*np.sum(G[li, :j] + 2*G2[li, :j])
            
            # increment Gm2
            Gm2[li, i+nsteps, j] = G2[li, j]
            Gm2[li, i+1+nsteps:j+1+nsteps, j] = Gm2[pli, i+nsteps:j+nsteps, j-1]
            
            # increment I3 (2 Gmt3 - Gm3 - 2 Gm4 + Gm5 + 3 Gm6 - Gm2)(0 4 -4 2 6 -4 -6)
            I3[li, j] = I3[pli, j-1]
            I3[li, j] += d*np.sum(2*Gm3t[pli, i+nsteps:j-1+nsteps, j-1] - Gm3[pli, i+nsteps:j-1+nsteps, j-1] - 2*Gm4[pli, i+nsteps:j-1+nsteps, j-1] + Gm5[pli, i+nsteps:j-1+nsteps, j-1] + 3*Gm6[pli, i+nsteps:j-1+nsteps, j-1] - Gm2[pli, i+nsteps:j-1+nsteps, j-1])
            I3[li, j] += 0.5*d*np.sum(4*I3t[li, :j] - 4*I4[li, :j] + 2*I5[li, :j] + 6*I6[li, :j] - 4*G[li, :j] - 6*G2[li, :j])
            
            # increment Gm3 
            Gm3[li, i+nsteps, j] = I3[li, j]
            for k in range(i+1, j+1):
                Gm3[li, k+nsteps, j] = Gm3[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm3[li, k+nsteps, j] += d*np.sum(2*Gm3t[pli, lb+nsteps:nsteps+j-1, j-1] - Gm3[pli, lb+nsteps:nsteps+j-1, j-1] - 2*Gm4[pli, lb+nsteps:nsteps+j-1, j-1] + Gm5[pli, lb+nsteps:nsteps+j-1, j-1] + 3*Gm6[pli, lb+nsteps:nsteps+j-1, j-1] - Gm2[pli, lb+nsteps:nsteps+j-1, j-1])

            # increment I3t (I3 tilde) (Gmt3 - (Nr/4) Gm3 + (Nr/2 - 2) Gm4 + (1-Nr/4)Gm5 + (3- 3Nr/4)Gm6 + (1-Nr/4)Gm2)
            # (-Nr/2 4 Nr-4 2-Nr/2 6-3Nr/2 -4 Nr/2-6)
            I3t[li, j] = I3t[pli, j-1] 
            I3t[li, j] += d*np.sum(Gm3t[pli, i+nsteps:j-1+nsteps, j-1] - (Nratio/4)*Gm3[pli, i+nsteps:j-1+nsteps, j-1]  + ((Nratio/2)-2)*Gm4[pli, i+nsteps:j-1+nsteps, j-1] + (1-(Nratio/4))*Gm5[pli, i+nsteps:j-1+nsteps, j-1] + (3 - ((3.0/4)*Nratio))*Gm6[pli, i+nsteps:j-1+nsteps, j-1] - (1- (Nratio/4))*Gm2[pli, i+nsteps:j-1+nsteps, j-1])
            I3t[li, j] += 0.5*d*np.sum(-(Nratio/2)*I3[li, :j] + 4*I3t[li, :j] + (Nratio-4)*I4[li, :j] + (2-(Nratio/2))*I5[li, :j] + (6-((3/2.0)*Nratio))*I6[li, :j] - 4*G[li, :j] + ((Nratio/2) - 6)*G2[li, :j])
                   
            # increment Gm3t (Gm3 tilde)
            Gm3t[li, i+nsteps, j] = I3t[li, j]
            for k in range(i+1, j+1):
                Gm3t[li, k+nsteps, j] = Gm3t[pli, k-1+nsteps, j-1]
                lb = max(i, k-1)
                Gm3t[li, k+nsteps, j] += d*np.sum(Gm3t[pli, lb+nsteps:nsteps+j-1, j-1] - (Nratio/4)*Gm3[pli, lb+nsteps:nsteps+j-1, j-1] + ((Nratio/2)-2)*Gm4[pli, lb+nsteps:nsteps+j-1, j-1] + (1-(Nratio/4))*Gm5[pli, lb+nsteps:nsteps+j-1, j-1] + (3 - (0.75*Nratio))*Gm6[pli, lb+nsteps:nsteps+j-1, j-1] - (1-(Nratio/4))*Gm2[pli, lb+nsteps:nsteps+j-1, j-1])

            # increment I4(0,1,0,4,2,-2)
            I4[li, j] = I4[pli, j-1]
            I4[li, j] += 0.5*d*np.sum(G2[li, :j] + 4*I4[li, :j] + 2*I5[li, :j] - 2*I6[li, :j])
            
            # increment Gm4
            Gm4[li, i+nsteps, j] = I4[li, j]
            Gm4[li, i+1+nsteps:j+1+nsteps, j] = Gm4[pli, i+nsteps:j+nsteps, j-1]
            
            # increment I5(2,3,-2,2,-1,-3)
            I5[li, j] = I5[pli, j-1]
            I5[li, j] += 0.5*d*np.sum(2*G[li, :j] + 3*G2[li, :j] - 2*I3t[li, :j] + 2*I4[li, :j] - I5[li, :j] - 3*I6[li, :j])
            
            # increment Gm5
            Gm5[li, i+nsteps, j] = I5[li, j]
            Gm5[li, i+1+nsteps:j+1+nsteps, j] = Gm5[pli, i+nsteps:j+nsteps, j-1]
            
            # increment I6(2,4,0,0,0,0)
            I6[li, j] = I6[pli, j-1]
            I6[li, j] += 0.5*d*np.sum(2*G[li, :j] + 4*G2[li, :j])
            
            # increment Gm6
            Gm6[li, i+nsteps, j] = I6[li, j]
            Gm6[li, i+1+nsteps:j+1+nsteps, j] = Gm6[pli, i+nsteps:j+nsteps, j-1]
            
        if(np.mod(j, 50) == 0) and verb: 
            now = datetime.now()
            curr_time = now.strftime("%H:%M.%S")
            print(str(round((j/round(nsteps))*100,1))+"% done ("+curr_time+")")
        
    return [Q, G, G2, I3, I3t, I4, I5, I6]